# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Lexicon #, Striplog, Decor, Legend 
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.io import boreholes_from_files,gdf_viewer, df_from_sources, gdf_merger
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 
from utils.lexicon.Lexicon_FR_updated import LEXICON
from os import remove
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR
import pandas as pd
import matplotlib.pyplot as plt

## Reading of CSV files in a dir

In [2]:
search_dir=ROOT_DIR+'/CF_data/Result_traitem/'
#cols=['ID','X','Y','Z','Long_Pz','Diam_Pz_int']
cols=None
filename='source_Boreholes'

In [3]:
df_all=df_from_sources(search_dir, filename, cols, verbose=False)

--> Prof_contact_sol_forage/source_Boreholes.csv: (8 lines)
--> Forage_Pilote/source_Boreholes.csv: (72 lines)
--> database_Memoris3/source_Boreholes.csv: (826 lines)
--> donnees_terrain_2019/source_Boreholes.csv: (16 lines)
--> profils_sols_donnees_forages/source_Boreholes.csv: (13 lines)

The overall dataframe contains 37 lines. 3 files used


In [4]:
cols=None
filename='source_Lithology'

df_litho=df_from_sources(search_dir, filename, cols, verbose=False)

--> vUmons_logsFor/source_Lithology.csv: (119 lines)
--> database_Memoris3/source_Lithology.csv: (7511 lines)
--> donnees_terrain_2019/source_Lithology.csv: (91 lines)
--> profils_sols_donnees_forages/source_Lithology.csv: (54 lines)

The overall dataframe contains 7630 lines. 2 files used


In [9]:
df_litho.merge(df_all, how='inner', on='ID')

,ID,X_x,Y_x,Z_x,Litho_top,Litho_base,Terrain,Refus,Date_ouv,Long_for_x,Z_fond,geometry_x,Long_for_y,X_y,Y_y,Z_y,Type,geometry_y
0,F205,152471.0070,122935.1210,108.778,0.0,0.6,Remblais,x,2010-03-26,NaN,NaN,POINT (152471.007 122935.121),3.2,152887.693,122594.620,101.804,Forage,POINT (152887.693 122594.620)
1,F205,152471.0070,122935.1210,108.778,0.0,0.6,Remblais,x,2010-03-26,NaN,NaN,POINT (152471.007 122935.121),3.2,152887.693,122594.620,101.804,Forage,POINT (152887.693 122594.620)
2,F205,152471.0070,122935.1210,108.778,0.6,1.4,Remblais,x,2010-03-26,NaN,NaN,POINT (152471.007 122935.121),3.2,152887.693,122594.620,101.804,Forage,POINT (152887.693 122594.620)
3,F205,152471.0070,122935.1210,108.778,0.6,1.4,Remblais,x,2010-03-26,NaN,NaN,POINT (152471.007 122935.121),3.2,152887.693,122594.620,101.804,Forage,POINT (152887.693 122594.620)
4,F212,152359.4035,122806.0686,105.218,0.0,0.1,Remblais,NaN,2010-05-10,NaN,NaN,POINT (152359.403 122806.069),3.4,152882.850,122591.453,101.930,Forage,POINT (152882.850 122591.453)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,F217,153133.8960,122458.6850,103.046,5.4,5.7,Remblais,x,2010-03-02,NaN,NaN,POINT (153133.896 122458.685),4.2,152886.185,122587.152,101.815,Forage,POINT (152886.185 122587.152)
102,F217,153133.8960,122458.6850,103.046,5.4,5.7,Remblais,x,2010-03-02,NaN,NaN,POINT (153133.896 122458.685),4.2,152886.185,122587.152,101.815,Forage,POINT (152886.185 122587.152)
103,F217,153133.8960,122458.6850,103.046,5.4,5.7,Argile,x,2010-03-02,NaN,NaN,POINT (153133.896 122458.685),4.2,152886.185,122587.152,101.815,Forage,POINT (152886.185 122587.152)
104,F217,153133.8960,122458.6850,103.046,5.4,5.7,Remblais,x,2010-03-02,NaN,NaN,POINT (153133.896 122458.685),4.2,152886.185,122587.152,101.815,Forage,POINT (152886.185 122587.152)


In [5]:
merge_df, error_df = gdf_merger(df_litho, df_all, col='ID', how='inner')

Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice
Mean of empty slice



Columns dropped :['X', 'Y', 'Z', 'Long_for_x', 'Z_fond', 'Long_for_y', 'geometry']

Ambiguous values in both columns compared, change it manually !
Columns ['ID', 'geometry_x', 'geometry_y', 'Y_x', 'Y_y', 'Z_x', 'Z_y', 'X_x', 'X_y'] must be dropped manually !
error file created in 'tmp_files/merging_error_log(-).csv'


In [6]:
merge_df = merge_df[['ID','X','Y','Z','Long_Pz','Diam_Pz_int','Description','Litho_top','Litho_base']]
gdf_viewer(merge_df, rows=5)

KeyError: "['Diam_Pz_int', 'Y', 'Z', 'Long_Pz', 'X', 'Description'] not in index"

## Data Checking

In [ ]:
dist_crit=1
df=merge_df

indexes = []
for i in range(0,len(df)-1):
    if 'geometry' in df.columns:
        d=df.loc[i, 'geometry'].distance(df.loc[i+1, 'geometry'])
        if d<dist_crit or d>1e+4:
            indexes.append(i)
            #print(f"{df.loc[i,'ID']} <-> {df.loc[i+1,'ID']} : {d:.2} meters")

if 'geometry' in df.columns:
    print('\n',f'{len(indexes)}/{len(df)} elements with probably bad XY')
    print(f'\nIndexes: {indexes}\n\nNAN_indexes: {df.query("X!=X").index}')
else:
    print('No geometry column in the dataframe')

## Lithology extraction and test

In [ ]:
lexicon = Lexicon(LEXICON)

In [ ]:
desc_col = 'Description'
df = merge_df

In [ ]:
comp_list = []
for d in df[desc_col]:
    comp_list.append(Component.from_text(d, lexicon))
df['Component']=comp_list

In [ ]:
for i in range(len(df)):
    d=df.loc[i, desc_col]
    Component.from_text(d, lexicon)

In [ ]:
idx=65

print(f"{type(df['Component'][idx])}\n\n idx {idx}: [{df['ID'][idx]}] --> "
      f"{df['Description'][idx]}")
df['Component'][idx] # TODO : need some modifications on colour, grainsize lexicons

In [ ]:
# problem with this description above : there are two intervals in one description --> 
# data incoherence (unfortunately)

## Create boreholes and components objects from dataframe

In [ ]:
boreholes, components, link_intv_comp= boreholes_from_files([df], verbose=False, diam_field='Diam_Pz_int', 
                                            length_field='Long_Pz', litho_field='Description',
                                           litho_top='Litho_top', litho_base='Litho_base',
                                           lexicon=lexicon)#, x=list(df_all.X), y=list(df_all.Y), verbose=False)

In [ ]:
len(boreholes), len(components)

### Saving in a database

In [ ]:
remove('tmp_files/tfe_orm_db.db')

In [ ]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [ ]:
Base.metadata.create_all(engine)

## Add differents objects into the project

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
p = Project(session)
p.add_components(components)

In [ ]:
for bh in boreholes:
    p.add_borehole(bh)

In [ ]:
p.add_link_between_components_and_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

In [ ]:
p.refresh(verbose=True)

In [ ]:
session.close()

In [ ]:
p.boreholes_3d[0].intervals

## The Borehole 3D plot using X3D for interaction within the notebook

In [ ]:
p.plot3d(x3d=True)

##  ================== Draft =================

#### Codes qui peuvent aider si besoin (Brouillon)